In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy

# Combinare opzioni di mitigazione degli errori con la primitiva Estimator

*Stima di utilizzo: Sette minuti su un processore Heron r2 (NOTA: Questa è solo una stima. Il tuo tempo di esecuzione potrebbe variare.)*

## Contesto
Questa guida esplora le opzioni di soppressione e mitigazione degli errori disponibili con la primitiva Estimator di Qiskit Runtime. Costruirai un circuito e un osservabile e sottometterai job utilizzando la primitiva Estimator con diverse combinazioni di impostazioni di mitigazione degli errori. Quindi, tracerai i risultati per osservare gli effetti delle varie impostazioni. La maggior parte degli esempi utilizza un circuito a 10 qubit per rendere più facili le visualizzazioni, e alla fine potrai scalare il workflow fino a 50 qubit.

Queste sono le opzioni di soppressione e mitigazione degli errori che utilizzerai:

- Disaccoppiamento dinamico
- Mitigazione degli errori di misura
- Gate twirling
- Estrapolazione a rumore zero (ZNE)
## Requisiti
Prima di iniziare questa guida, assicurati di avere installato quanto segue:

- Qiskit SDK v2.1 o successivo, con supporto per la [visualizzazione](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.40 o successivo (`pip install qiskit-ibm-runtime`)
## Setup

In [7]:
import matplotlib.pyplot as plt
import numpy as np

from qiskit.circuit.library import efficient_su2, unitary_overlap
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Batch, EstimatorV2 as Estimator

## Passo 1: Mappare input classici a un problema quantistico
Questa guida assume che il problema classico sia già stato mappato in forma quantistica. Inizia costruendo un circuito e un osservabile da misurare. Sebbene le tecniche qui utilizzate si applichino a molti tipi diversi di circuiti, per semplicità questa guida utilizza il circuito [`efficient_su2`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.efficient_su2) incluso nella libreria di circuiti di Qiskit.

`efficient_su2` è un circuito quantistico parametrizzato progettato per essere eseguito in modo efficiente su hardware quantistico con connettività limitata dei qubit, pur essendo sufficientemente espressivo per risolvere problemi in domini applicativi come l'ottimizzazione e la chimica. È costruito alternando strati di gate parametrizzati a singolo qubit con uno strato contenente un pattern fisso di gate a due qubit, per un numero scelto di ripetizioni. Il pattern di gate a due qubit può essere specificato dall'utente. Qui puoi utilizzare il pattern integrato `pairwise` perché minimizza la profondità del circuito impacchettando i gate a due qubit nel modo più denso possibile. Questo pattern può essere eseguito utilizzando solo connettività lineare dei qubit.

In [4]:
n_qubits = 10
reps = 1

circuit = efficient_su2(n_qubits, entanglement="pairwise", reps=reps)

circuit.decompose().draw("mpl", scale=0.7)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/24abd7ba-bbb8-443b-9e81-866795d39a6c-1.avif)

Per il nostro osservabile, prendiamo l'operatore di Pauli $Z$ che agisce sull'ultimo qubit, $Z I \cdots I$.

In [5]:
# Z on the last qubit (index -1) with coefficient 1.0
observable = SparsePauliOp.from_sparse_list(
    [("Z", [-1], 1.0)], num_qubits=n_qubits
)

A questo punto, potresti procedere a eseguire il tuo circuito e misurare l'osservabile. Tuttavia, vuoi anche confrontare l'output del dispositivo quantistico con la risposta corretta - cioè, il valore teorico dell'osservabile, se il circuito fosse stato eseguito senza errori. Per circuiti quantistici piccoli puoi calcolare questo valore simulando il circuito su un computer classico, ma questo non è possibile per circuiti più grandi, su scala di utilità. Puoi aggirare questo problema con la tecnica del "circuito specchiato" (nota anche come "compute-uncompute"), che è utile per il benchmarking delle prestazioni dei dispositivi quantistici.

#### Circuito specchiato
Nella tecnica del circuito specchiato, concatena il circuito con il suo circuito inverso, che è formato invertendo ciascun gate del circuito in ordine inverso. Il circuito risultante implementa l'operatore identità, che può essere simulato in modo banale. Poiché la struttura del circuito originale è preservata nel circuito specchiato, eseguire il circuito specchiato dà comunque un'idea di come il dispositivo quantistico si comporterebbe sul circuito originale.

La seguente cella di codice assegna parametri casuali al tuo circuito, e quindi costruisce il circuito specchiato utilizzando la classe [`unitary_overlap`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.unitary_overlap). Prima di specchiare il circuito, aggiungi un'istruzione [barrier](https://docs.quantum.ibm.com/api/qiskit/circuit#qiskit.circuit.Barrier) ad esso per impedire al transpiler di fondere le due parti del circuito su entrambi i lati della barriera. Senza la barriera, il transpiler fonderebbe il circuito originale con il suo inverso, risultando in un circuito traspilato senza alcun gate.

In [8]:
# Generate random parameters
rng = np.random.default_rng(1234)
params = rng.uniform(-np.pi, np.pi, size=circuit.num_parameters)

# Assign the parameters to the circuit
assigned_circuit = circuit.assign_parameters(params)

# Add a barrier to prevent circuit optimization of mirrored operators
assigned_circuit.barrier()

# Construct mirror circuit
mirror_circuit = unitary_overlap(assigned_circuit, assigned_circuit)

mirror_circuit.decompose().draw("mpl", scale=0.7)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/4dbde811-1ba9-47a8-85a0-dcaff054ed60-1.avif)

## Passo 2: Ottimizzare il problema per l'esecuzione su hardware quantistico
Devi ottimizzare il tuo circuito prima di eseguirlo sull'hardware. Questo processo comporta alcuni passi:

- Scegliere un layout di qubit che mappi i qubit virtuali del tuo circuito ai qubit fisici sull'hardware.
- Inserire gate di swap secondo necessità per instradare interazioni tra qubit che non sono connessi.
- Tradurre i gate nel tuo circuito in istruzioni [Instruction Set Architecture (ISA)](/guides/transpile#instruction-set-architecture) che possono essere eseguite direttamente sull'hardware.
- Eseguire ottimizzazioni del circuito per minimizzare la profondità del circuito e il conteggio dei gate.

Il transpiler integrato in Qiskit può eseguire tutti questi passi per te. Poiché questo esempio utilizza un circuito hardware-efficient, il transpiler dovrebbe essere in grado di scegliere un layout di qubit che non richieda l'inserimento di alcun gate di swap per l'instradamento delle interazioni.

Devi scegliere il dispositivo hardware da utilizzare prima di ottimizzare il tuo circuito. La seguente cella di codice richiede il dispositivo meno occupato con almeno 127 qubit.

In [9]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=127
)

Puoi traspilare il tuo circuito per il tuo backend scelto creando un pass manager e quindi eseguendo il pass manager sul circuito. Un modo semplice per creare un pass manager è utilizzare la funzione [`generate_preset_pass_manager`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.generate_preset_pass_manager). Consulta [Transpile with pass managers](/guides/transpile-with-pass-managers) per una spiegazione più dettagliata della traspilazione con i pass manager.

In [10]:
pass_manager = generate_preset_pass_manager(
    optimization_level=3, backend=backend, seed_transpiler=1234
)
isa_circuit = pass_manager.run(mirror_circuit)

isa_circuit.draw("mpl", idle_wires=False, scale=0.7, fold=-1)

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-0.avif" alt="Output of the previous code cell" />

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-0.avif)

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/027e829a-44d3-455e-b2bf-8ce0d7e26b9b-1.avif)

Il circuito traspilato ora contiene solo istruzioni ISA. I gate a singolo qubit sono stati decomposti in termini di gate $\sqrt{X}$ e rotazioni $R_z$, e i gate CX sono stati decomposti in [gate ECR](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ECRGate#ecrgate) e rotazioni a singolo qubit.

Il processo di traspilazione ha mappato i qubit virtuali del circuito ai qubit fisici sull'hardware. Le informazioni sul layout dei qubit sono memorizzate nell'attributo `layout` del circuito traspilato. Anche l'osservabile è stato definito in termini di qubit virtuali, quindi devi applicare questo layout all'osservabile, cosa che puoi fare con il metodo [`apply_layout`](https://docs.quantum.ibm.com/api/qiskit/qiskit.quantum_info.SparsePauliOp#apply_layout) di `SparsePauliOp`.

In [12]:
isa_observable = observable.apply_layout(isa_circuit.layout)

print("Original observable:")
print(observable)
print()
print("Observable with layout applied:")
print(isa_observable)

Original observable:
SparsePauliOp(['ZIIIIIIIII'],
              coeffs=[1.+0.j])

Observable with layout applied:
SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


## Step 3: Execute using Qiskit primitives

You are now ready to run your circuit using the Estimator primitive.

Here you will submit five separate jobs, starting with no error suppression or mitigation, and successively enabling various error suppression and mitigation options available in Qiskit Runtime. For information about the options, refer to the following pages:

- [Overview of all options](/docs/api/qiskit-ibm-runtime/options)
- [Dynamical decoupling](/docs/api/qiskit-ibm-runtime/options-dynamical-decoupling-options)
- [Resilience, including measurement error mitigation and zero-noise extrapolation (ZNE)](/docs/api/qiskit-ibm-runtime/options-resilience-options-v2)
- [Twirling](/docs/api/qiskit-ibm-runtime/options-twirling-options)

Because these jobs can run independently of each other, you can use [batch mode](/docs/guides/run-jobs-batch) to allow Qiskit Runtime to optimize the timing of their execution.

In [13]:
pub = (isa_circuit, isa_observable)

jobs = []

with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    # Set number of shots
    estimator.options.default_shots = 100_000
    # Disable runtime compilation and error mitigation
    estimator.options.resilience_level = 0

    # Run job with no error mitigation
    job0 = estimator.run([pub])
    jobs.append(job0)

    # Add dynamical decoupling (DD)
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"
    job1 = estimator.run([pub])
    jobs.append(job1)

    # Add readout error mitigation (DD + TREX)
    estimator.options.resilience.measure_mitigation = True
    job2 = estimator.run([pub])
    jobs.append(job2)

    # Add gate twirling (DD + TREX + Gate Twirling)
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"
    job3 = estimator.run([pub])
    jobs.append(job3)

    # Add zero-noise extrapolation (DD + TREX + Gate Twirling + ZNE)
    estimator.options.resilience.zne_mitigation = True
    estimator.options.resilience.zne.noise_factors = (1, 3, 5)
    estimator.options.resilience.zne.extrapolator = ("exponential", "linear")
    job4 = estimator.run([pub])
    jobs.append(job4)

## Passo 3: Eseguire utilizzando le primitive Qiskit
Ora sei pronto a eseguire il tuo circuito utilizzando la primitiva Estimator.

Qui sottometterai cinque job separati, iniziando senza soppressione o mitigazione degli errori, e abilitando successivamente varie opzioni di soppressione e mitigazione degli errori disponibili in Qiskit Runtime. Per informazioni sulle opzioni, fai riferimento alle seguenti pagine:

- [Panoramica di tutte le opzioni](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options)
- [Disaccoppiamento dinamico](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-dynamical-decoupling-options)
- [Resilienza, inclusa la mitigazione degli errori di misura e l'estrapolazione a rumore zero (ZNE)](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-resilience-options-v2)
- [Twirling](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/options-twirling-options)

Poiché questi job possono essere eseguiti indipendentemente l'uno dall'altro, puoi utilizzare la [modalità batch](/guides/run-jobs-batch) per consentire a Qiskit Runtime di ottimizzare la temporizzazione della loro esecuzione.

In [14]:
# Retrieve the job results
results = [job.result() for job in jobs]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array(
    [float(pub_result.data.evs) for pub_result in pub_results]
)
standard_errors = np.array(
    [float(pub_result.data.stds) for pub_result in pub_results]
)

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation", "+ DD", "+ TREX", "+ Twirling", "+ ZNE"]
ax.bar(
    range(len(labels)),
    expectation_vals,
    yerr=standard_errors,
    label="experiment",
)
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/eef38976-0ca2-429a-b2dc-41aac69605f7-0.avif" alt="Output of the previous code cell" />

## Passo 4: Post-elaborare e restituire il risultato nel formato classico desiderato
Infine, puoi analizzare i dati. Qui recupererai i risultati dei job, estrarrai i valori di aspettazione misurati da essi e tracerai i valori, includendo barre di errore di una deviazione standard.

In [15]:
n_qubits = 50
reps = 1

# Construct circuit and observable
circuit = efficient_su2(n_qubits, entanglement="pairwise", reps=reps)
observable = SparsePauliOp.from_sparse_list(
    [("Z", [-1], 1.0)], num_qubits=n_qubits
)

# Assign parameters to circuit
params = rng.uniform(-np.pi, np.pi, size=circuit.num_parameters)
assigned_circuit = circuit.assign_parameters(params)
assigned_circuit.barrier()

# Construct mirror circuit
mirror_circuit = unitary_overlap(assigned_circuit, assigned_circuit)

# Transpile circuit and observable
isa_circuit = pass_manager.run(mirror_circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)

# Run jobs
pub = (isa_circuit, isa_observable)

jobs = []

with Batch(backend=backend) as batch:
    estimator = Estimator(mode=batch)
    # Set number of shots
    estimator.options.default_shots = 100_000
    # Disable runtime compilation and error mitigation
    estimator.options.resilience_level = 0

    # Run job with no error mitigation
    job0 = estimator.run([pub])
    jobs.append(job0)

    # Add dynamical decoupling (DD)
    estimator.options.dynamical_decoupling.enable = True
    estimator.options.dynamical_decoupling.sequence_type = "XpXm"
    job1 = estimator.run([pub])
    jobs.append(job1)

    # Add readout error mitigation (DD + TREX)
    estimator.options.resilience.measure_mitigation = True
    job2 = estimator.run([pub])
    jobs.append(job2)

    # Add gate twirling (DD + TREX + Gate Twirling)
    estimator.options.twirling.enable_gates = True
    estimator.options.twirling.num_randomizations = "auto"
    job3 = estimator.run([pub])
    jobs.append(job3)

    # Add zero-noise extrapolation (DD + TREX + Gate Twirling + ZNE)
    estimator.options.resilience.zne_mitigation = True
    estimator.options.resilience.zne.noise_factors = (1, 3, 5)
    estimator.options.resilience.zne.extrapolator = ("exponential", "linear")
    job4 = estimator.run([pub])
    jobs.append(job4)

# Retrieve the job results
results = [job.result() for job in jobs]

# Unpack the PUB results (there's only one PUB result in each job result)
pub_results = [result[0] for result in results]

# Unpack the expectation values and standard errors
expectation_vals = np.array(
    [float(pub_result.data.evs) for pub_result in pub_results]
)
standard_errors = np.array(
    [float(pub_result.data.stds) for pub_result in pub_results]
)

# Plot the expectation values
fig, ax = plt.subplots()
labels = ["No mitigation", "+ DD", "+ TREX", "+ Twirling", "+ ZNE"]
ax.bar(
    range(len(labels)),
    expectation_vals,
    yerr=standard_errors,
    label="experiment",
)
ax.axhline(y=1.0, color="gray", linestyle="--", label="ideal")
ax.set_xticks(range(len(labels)))
ax.set_xticklabels(labels)
ax.set_ylabel("Expectation value")
ax.legend(loc="upper left")

plt.show()

<Image src="../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/d7d8408b-faf1-4eda-ab9c-bdeaab01ff53-0.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/combine-error-mitigation-techniques/extracted-outputs/eef38976-0ca2-429a-b2dc-41aac69605f7-0.avif)

A questa piccola scala, è difficile vedere l'effetto della maggior parte delle tecniche di mitigazione degli errori, ma l'estrapolazione a rumore zero fornisce un miglioramento evidente. Tuttavia, nota che questo miglioramento non arriva gratuitamente, perché il risultato ZNE ha anche una barra di errore più grande.
## Scalare l'esperimento
Quando si sviluppa un esperimento, è utile iniziare con un circuito piccolo per rendere più facili le visualizzazioni e le simulazioni. Ora che hai sviluppato e testato il workflow su un circuito a 10 qubit, puoi scalarlo fino a 50 qubit. La seguente cella di codice ripete tutti i passi di questa guida, ma ora li applica a un circuito a 50 qubit.